In [ ]:
import torch
import torch.nn.functional as F
from torch import Tensor

import numpy as np
import os
import tifffile
import scipy.io

## For generating the traditional frame as label

In [ ]:
up_sample = Tensor([4, 4, 4]).int() 
dim_label = Tensor([128, 2048, 2048]).int() 
lum_info  = False

# data path
frames_load_fold = "D:/frames"
mlists_load_fold = "D:/mlists"

# file name list
frames_list = os.listdir(frames_load_fold)
mlists_list = os.listdir(mlists_load_fold)

result = torch.zeros(*dim_label.tolist())
for index in range(45000):
    # frame
    frame = torch.from_numpy(tifffile.imread(
        os.path.join(frames_load_fold, frames_list[index])
    ))
    frame = (frame / 6.5).float()
    frame = F.interpolate(
        frame.unsqueeze(0).unsqueeze(0), 
        size = (32, 512, 512)
    ).squeeze(0).squeeze(0)
    frame = torch.clip(frame, 0, 1)

    frame = F.interpolate(
        frame.unsqueeze(0).unsqueeze(0), 
        scale_factor=up_sample.tolist()
    ).squeeze(0).squeeze(0)

    # mlist
    _, mlist = scipy.io.loadmat(
        os.path.join(mlists_load_fold, mlists_list[index])
    ).popitem()
    mlist = torch.from_numpy(mlist).float()
    mlist = mlist[:, [2, 0, 1]] - 1  # (H W D) -> (D H W)
    mlist[:, 0] = (mlist[:, 0] + 0.5) / 2 - 0.5

    mlist = (mlist + 0.5) * up_sample - 0.5
    mlist = torch.round(mlist).int()

    # label
    label = torch.zeros(*dim_label.tolist())
    label[tuple(mlist.t())] = 1
    if lum_info: label *= frame
    result += label

    # save after combine 1, 2, 4, 8, 16... frames
    current_frame = index + 1
    if current_frame & (current_frame - 1) == 0 or index == 45000 - 1:
        tifffile.imwrite(
            "{}/{:05}.tif".format("temp", current_frame),
            result.numpy()
        )

## For concat threshold eval result, d_06

In [ ]:
for s in [
    "000",
    "010", "020", "030", "040", "050", 
    "060", "070", "080", "090", "100",
]:
    # T Z Y X
    T = []
    for t in [
        1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 
        1024, 2048, 4096, 8192, 16384, 32768, 45000
    ]: T.append(tifffile.imread(
        "data/d_06/{}/outputs_{}.tif".format(s, t)
    ))

    # Z T Y X
    T = np.array(T).transpose(1, 0, 2, 3)

    T[T > 10.0] = 10.0

    tifffile.imwrite(
        "C:/Users/tianrui/Desktop/{}.tif".format(s), 
        np.array(T), imagej=True,
        metadata={
            "unit": "nm",
            "axes": "TZYX",
        }
    )

In [ ]:
# Z T Y X
Z = []
for s in [
    "000",
    "010", "020", "030", "040", "050", 
    "060", "070", "080", "090", "100",
]: Z.append(tifffile.imread(
    "C:/Users/tianrui/Desktop/{}_h.tif".format(s)
))

# T Z Y X
Z = np.array(Z).transpose(1, 0, 2, 3, 4)

tifffile.imwrite(
    "C:/Users/tianrui/Desktop/threshold.tif", 
    np.array(Z), imagej=True,
    metadata={
        "unit": "nm",
        "axes": "TZYXS",
    }
)